In [1]:
from mmdet3d.apis import LidarDet3DInferencer

In [2]:
# initialize inferencer
inferencer = LidarDet3DInferencer('pointpillars_donaset-car')
# inferencer = LidarDet3DInferencer("pointpillars_hv_secfpn_8xb6-160e_kittilidar-3d-car",scope="mmdet3d")

Loads checkpoint by local backend from path: /home/riccardo/LidarObjDetection/mmdetection3d_dona/mmdetection3d/work_dirs/pointpillars_hv_secfpn_8xb6-160e_donaset-3d-car/iter_2500.pth
02/16 15:36:28 - mmengine - WARNING - Failed to search registry with scope "mmdet3d" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet3d" is a correct scope, or whether the registry is initialized.


/home/riccardo/LidarObjDetection/mmdetection3d_dona/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/riccardo/venvs/pytorch_cuda123_env/lib/python3.10/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [19]:
import inspect
args = inspect.getfullargspec(inferencer.visualize).args
args

['self',
 'inputs',
 'preds',
 'return_vis',
 'show',
 'wait_time',
 'draw_pred',
 'pred_score_thr',
 'no_save_vis',
 'img_out_dir']

In [4]:
# inference
pcl = '../data/DonaSet/testing/velodyne/000001.bin'
# pcl = './data/falcon/falcon1.bin'

inputs = dict(points=pcl)
# inferencer(inputs, show=False,pred_score_thr=0.3, batch_size=4)

In [11]:
import time
start = time.time()
inferencer(inputs, show=False,pred_score_thr=0.3)
end = time.time()
print("Time Taken {}".format(end-start))

Time Taken 257.9369606971741


In [6]:
# Check the device of the model
device = next(inferencer.model.parameters()).device

if device.type == 'cuda':
    print("Model is on GPU")
else:
    print("Model is on CPU")

Model is on GPU


In [ ]:
# If your operating environment does not have a display device,
# (e.g. a remote server), you can save the predictions and visualize
# them in local devices.
inferencer(inputs, show=False, out_dir='./remote_outputs')

# Simulate the migration process
%mv ./remote_outputs ./local_outputs

# Visualize the predictions from the saved files
# NOTE: use the `Esc` key to exit Open3D window in Jupyter Notebook Environment
local_inferencer = LidarDet3DInferencer('pointpillars_kitti-3class')
inputs = local_inferencer._inputs_to_list(inputs)
local_inferencer.visualize_preds_fromfile(inputs, ['local_outputs/preds/000008.json'], show=True)

Model Analysis

In [15]:
model = inferencer.model

In [50]:
# Count the number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

# Access and print model parameters
for name, param in model.named_parameters():
    print(f"Parameter: {name}, Size: {param.size()}")

Total parameters: 4814868
Parameter: backbone.blocks.0.0.weight, Size: torch.Size([64, 64, 3, 3])
Parameter: backbone.blocks.0.1.weight, Size: torch.Size([64])
Parameter: backbone.blocks.0.1.bias, Size: torch.Size([64])
Parameter: backbone.blocks.0.3.weight, Size: torch.Size([64, 64, 3, 3])
Parameter: backbone.blocks.0.4.weight, Size: torch.Size([64])
Parameter: backbone.blocks.0.4.bias, Size: torch.Size([64])
Parameter: backbone.blocks.0.6.weight, Size: torch.Size([64, 64, 3, 3])
Parameter: backbone.blocks.0.7.weight, Size: torch.Size([64])
Parameter: backbone.blocks.0.7.bias, Size: torch.Size([64])
Parameter: backbone.blocks.0.9.weight, Size: torch.Size([64, 64, 3, 3])
Parameter: backbone.blocks.0.10.weight, Size: torch.Size([64])
Parameter: backbone.blocks.0.10.bias, Size: torch.Size([64])
Parameter: backbone.blocks.1.0.weight, Size: torch.Size([128, 64, 3, 3])
Parameter: backbone.blocks.1.1.weight, Size: torch.Size([128])
Parameter: backbone.blocks.1.1.bias, Size: torch.Size([128])

In [15]:
import torch
# Evaluate on validation data
model.eval()
with torch.no_grad():
    val_output = model(inputs)
    # val_loss = loss_function(val_output, val_target)


KeyError: 'voxels'

In [13]:
import numpy as np
path = '../data/DonaSet/testing/velodyne/000001.bin'

fields = 4
# Read the binary file into a NumPy array
data = np.fromfile(path, dtype=np.float32)

# Reshape the 1D array to a 2D array with 4 columns
data = data.reshape(-1, 4)
data[0]

array([ 17.639317  , -13.64231   ,   0.98981375,   0.        ],
      dtype=float32)

In [20]:
import torch 
from mmdet3d.structures import  Det3DDataSample
# Assuming you have a single sample as a NumPy array or list

# Convert the sample to a PyTorch tensor
input_tensor = torch.tensor(data, dtype=torch.float32)

# Add batch dimension (if needed)
input_tensor = input_tensor  # For models expecting a batch dimension

# Set the model to evaluation mode
model.eval()

model_input = {}
model_input["inputs"] = {}
model_input["inputs"]["points"] = [input_tensor]
data_sample = Det3DDataSample()
model_input["data_samples"] = [data_sample]
# Perform inference
with torch.no_grad():
    output = model.test_step(model_input)

KeyError: 'voxels'

In [53]:
# Print the input size of the first layer
input_size = next(model.parameters()).size()[1:]
print("Input size:", input_size)

Input size: torch.Size([64, 3, 3])


In [24]:
from mmdet3d.registry import DATASETS, MODELS
MODELS.build(mod)

/home/riccardo/LidarObjDetection/mmdetection3d_dona/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


AttributeError: 'dict' object has no attribute 'assigner'